<a href="https://colab.research.google.com/github/Jimmynycu/finrl_for_VICI/blob/main/FINRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Generate a Python script for Google Colab using the FINRL library to train a Deep Reinforcement Learning agent for trading TSMC stock, following the specified workflow, including environment setup, data downloading, feature engineering, environment creation, agent training, and backtesting, with all configurations centralized in a dictionary and extensive comments.

## Environment setup

### Subtask:
Install necessary libraries using `!pip install`.


**Reasoning**:
The subtask requires executing the provided code cell to install the necessary libraries.



In [1]:
!pip install --quiet swig
!pip install --quiet wrds
!pip install --quiet pyportfolioopt
!pip install --quiet git+https://github.com/AI4Finance-Foundation/FinRL.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 15.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.7/108.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━

## Library imports and configuration

### Subtask:
Import required libraries and define a configuration dictionary with all parameters.


**Reasoning**:
Import the necessary libraries and define the configuration dictionary as instructed.



In [15]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

# Define the configuration dictionary
config = {
    'TICKER_LIST': ["TSM"],
    # --- ADJUST THESE DATES ---
    'START_DATE': '2015-01-01',
    'END_DATE': '2025-08-26',        # Extend data download to yesterday
    'INDICATORS': INDICATORS,

    'TRAIN_START_DATE': '2015-01-01',
    'TRAIN_END_DATE': '2022-12-31',    # Train on 8 full years of data
    'TRADE_START_DATE': '2023-01-01',
    'TRADE_END_DATE': '2025-08-26',
    'ERL_PARAMS': {
        'learning_rate': 1e-5,
        'batch_size': 128,
        'gamma': 0.99,
        'seed': 312,
        'net_dimension': 512,
        'target_tau': 0.001,
        'activation_fn': 'relu',
        'use_batch_norm': False,
        'use_layer_norm': False,
        'state_memory_size': 100,
        'buffer_size': 1000000,
        'train_freq': 1,
        'gradient_steps': 1,
        'ent_coef': 0.01,
        'action_noise': None,
        'optimize_memory_usage': False,
        'policy_kwargs': None,
        'device': 'auto'
    },
    'AGENT': 'ppo',
    'TRAIN_STEPS': 200000
}

# Display the config dictionary to verify
display(config)

{'TICKER_LIST': ['TSM'],
 'START_DATE': '2015-01-01',
 'END_DATE': '2025-08-26',
 'INDICATORS': ['macd',
  'boll_ub',
  'boll_lb',
  'rsi_30',
  'cci_30',
  'dx_30',
  'close_30_sma',
  'close_60_sma'],
 'TRAIN_START_DATE': '2015-01-01',
 'TRAIN_END_DATE': '2022-12-31',
 'TRADE_START_DATE': '2023-01-01',
 'TRADE_END_DATE': '2025-08-26',
 'ERL_PARAMS': {'learning_rate': 1e-05,
  'batch_size': 128,
  'gamma': 0.99,
  'seed': 312,
  'net_dimension': 512,
  'target_tau': 0.001,
  'activation_fn': 'relu',
  'use_batch_norm': False,
  'use_layer_norm': False,
  'state_memory_size': 100,
  'buffer_size': 1000000,
  'train_freq': 1,
  'gradient_steps': 1,
  'ent_coef': 0.01,
  'action_noise': None,
  'optimize_memory_usage': False,
  'policy_kwargs': None,
  'device': 'auto'},
 'AGENT': 'ppo',
 'TRAIN_STEPS': 200000}

## Data downloading

### Subtask:
Download historical stock data for TSMC using `YahooDownloader` based on the dates in the configuration.


**Reasoning**:
Download the historical stock data for TSMC using the YahooDownloader based on the dates and ticker in the config dictionary, and display the head and shape of the resulting dataframe.



In [16]:
# Data Downloading Cell

# Instantiate YahooDownloader with parameters from the config dictionary
downloader = YahooDownloader(start_date=config['START_DATE'],
                             end_date=config['END_DATE'],
                             ticker_list=config['TICKER_LIST'])

# Fetch the data
df = downloader.fetch_data()

# Display the first few rows and the shape of the dataframe
display(df.head())
display(df.shape)

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (2677, 8)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Price,date,close,high,low,open,volume,tic,day
0,2015-01-02,16.729322,16.864477,16.519079,16.856970,6074100,TSM,4
1,2015-01-05,16.323853,16.646726,16.301326,16.624199,9031800,TSM,0
2,2015-01-06,16.038525,16.203715,15.888351,16.008489,10169500,TSM,1
3,2015-01-07,16.271296,16.338874,16.023509,16.211226,10180200,TSM,2
4,2015-01-08,16.376413,16.526587,16.271292,16.286310,15825900,TSM,3


(2677, 8)

In [17]:
# --- Data Sanity Check Cell ---
# Run this cell immediately after downloading the data

print("--- Inspecting Raw Downloaded Data ---")

# Check the first 5 and last 5 rows to verify the date range
print("First 5 rows:")
display(df.head())
print("\nLast 5 rows:")
display(df.tail())

# Get a summary of the data types and check for missing values
print("\nDataFrame Info (non-null counts):")
df.info()

# Get a statistical summary of the data
print("\nStatistical Summary:")
display(df.describe())

--- Inspecting Raw Downloaded Data ---
First 5 rows:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Price,date,close,high,low,open,volume,tic,day
0,2015-01-02,16.729322,16.864477,16.519079,16.856970,6074100,TSM,4
1,2015-01-05,16.323853,16.646726,16.301326,16.624199,9031800,TSM,0
2,2015-01-06,16.038525,16.203715,15.888351,16.008489,10169500,TSM,1
3,2015-01-07,16.271296,16.338874,16.023509,16.211226,10180200,TSM,2
4,2015-01-08,16.376413,16.526587,16.271292,16.286310,15825900,TSM,3



Last 5 rows:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Price,date,close,high,low,open,volume,tic,day
2672,2025-08-19,232.699997,240.169998,232.580002,240.020004,14594700,TSM,1
2673,2025-08-20,228.600006,229.029999,223.699997,228.139999,17165200,TSM,2
2674,2025-08-21,227.330002,230.330002,226.259995,228.149994,7449100,TSM,3
2675,2025-08-22,232.990005,234.449997,226.169998,228.000000,10299500,TSM,4
2676,2025-08-25,235.589996,237.279999,232.250000,234.300003,7655000,TSM,0



DataFrame Info (non-null counts):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2677 entries, 0 to 2676
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    2677 non-null   object 
 1   close   2677 non-null   float64
 2   high    2677 non-null   float64
 3   low     2677 non-null   float64
 4   open    2677 non-null   float64
 5   volume  2677 non-null   int64  
 6   tic     2677 non-null   object 
 7   day     2677 non-null   int32  
dtypes: float64(4), int32(1), int64(1), object(2)
memory usage: 157.0+ KB

Statistical Summary:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Price,close,high,low,open,volume,day
count,2677.000000,2677.000000,2677.000000,2677.000000,2.677000e+03,2677.000000
mean,73.089556,73.956653,72.215278,73.121938,1.014170e+07,2.023907
std,54.827089,55.602160,54.045682,54.882874,5.841133e+06,1.399134
min,14.534039,15.053112,13.418420,13.743809,1.499700e+06,0.000000
25%,30.461178,30.602273,30.187269,30.328370,6.482600e+06,1.000000
50%,51.527184,51.824509,51.013618,51.482130,8.827200e+06,2.000000
75%,104.306305,105.536495,103.001385,104.215553,1.210640e+07,3.000000
max,245.600006,248.279999,241.699997,246.429993,6.866750e+07,4.000000


## Feature engineering

### Subtask:
Apply technical indicators and other features to the downloaded data using `FeatureEngineer`.


**Reasoning**:
The previous code failed because `FeatureEngineer` does not have an `add_tradable_day` method. The day of the week column is already present in the dataframe from the YahooDownloader. Also, the error indicates that the `add_turbulence` method also doesn't exist. I will remove the calls to these non-existent methods and only use `add_technical_indicator`.



In [18]:
# Feature Engineering Cell

# Import the original INDICATORS list from finrl.config
from finrl.config import INDICATORS as ORIGINAL_INDICATORS

# Instantiate FeatureEngineer with the original list of indicators
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = ORIGINAL_INDICATORS, # Use original indicator names
                     use_turbulence=False,
                     user_defined_feature=False)

# Add technical indicators
# IMPORTANT: Ensure you run this cell only ONCE after running the data downloading cell (6062dff2)
df = fe.add_technical_indicator(df)

# Fill NaN values with 0 after adding technical indicators
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Display the head and shape of the modified dataframe
display(df.head())
display(df.shape)

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2015-01-02,16.729322,16.864477,16.519079,16.856970,6074100,TSM,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16.729322,16.729322
1,2015-01-05,16.323853,16.646726,16.301326,16.624199,9031800,TSM,0,-0.009097,17.100009,15.953166,0.000000,-66.666667,100.000000,16.526587,16.526587
2,2015-01-06,16.038525,16.203715,15.888351,16.008489,10169500,TSM,1,-0.020439,17.058171,15.669628,0.000000,-100.000000,100.000000,16.363900,16.363900
3,2015-01-07,16.271296,16.338874,16.023509,16.211226,10180200,TSM,2,-0.016693,16.915133,15.766364,26.228389,-41.069222,62.036861,16.340749,16.340749
4,2015-01-08,16.376413,16.526587,16.271292,16.286310,15825900,TSM,3,-0.010123,16.846335,15.849428,34.280945,13.384439,26.413128,16.347882,16.347882


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


(2677, 16)

## Environment creation

### Subtask:
Create a custom trading environment compatible with Stable Baselines3 using the processed data.


**Reasoning**:
Create the custom trading environment using the processed data and the configuration parameters.



In [19]:
# Environment Creation Cell

# Import the StockTradingEnv class from the identified path
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv


from stable_baselines3.common.vec_env import DummyVecEnv

# Define environment parameters
stock_dimension = len(config['TICKER_LIST'])
state_space = 1 + 2*stock_dimension + len(config['INDICATORS'])
print(f"State space dimension: {state_space}")
# Define buy and sell costs (0.1% of the transaction amount)
# Since we have only one stock, the list will have one element
buy_cost_list = sell_cost_list = [0.001] * stock_dimension

# Fill any remaining NaN values with 0 just before splitting the data
df.fillna(0, inplace=True)


# Split the df DataFrame into training and trading datasets
train_data = data_split(df, config['TRAIN_START_DATE'], config['TRAIN_END_DATE'])
trade_data = data_split(df, config['TRADE_START_DATE'], config['TRADE_END_DATE'])

# Instantiate the StockTradingEnv for the training data
e_train_gym = StockTradingEnv(df = train_data,
                              stock_dim = stock_dimension,
                              hmax = 100, # Max shares to trade
                              initial_amount = 100000, # Starting cash
                              num_stock_shares = [0] * stock_dimension, # Add the num_stock_shares argument
                              state_space = state_space,
                              tech_indicator_list = config['INDICATORS'],
                              action_space = stock_dimension, # Action space is the number of stocks
                              buy_cost_pct = buy_cost_list,
                              sell_cost_pct = sell_cost_list,
                              reward_scaling = 1e-4, # Scale the reward
                              print_verbosity = 5 # Print frequency
                             )

# Wrap the training environment using DummyVecEnv
env_train = DummyVecEnv([lambda: e_train_gym])

# Instantiate the StockTradingEnv for the trading data
# Instantiate the StockTradingEnv for the trading data
e_trade_gym = StockTradingEnv(df = trade_data,
                              stock_dim = stock_dimension,
                              hmax = 100,
                              initial_amount = 100000,
                              # --- ADD THIS LINE ---
                              num_stock_shares = [0] * stock_dimension,
                              state_space = state_space,
                              tech_indicator_list = config['INDICATORS'],
                              action_space = stock_dimension,
                              buy_cost_pct = buy_cost_list,
                              sell_cost_pct = sell_cost_list,
                              reward_scaling = 1e-4,
                              print_verbosity = 5
                             )

# Wrap the trading environment using DummyVecEnv
env_trade = DummyVecEnv([lambda: e_trade_gym])

# Print a message indicating successful creation
print("Training and trading environments created successfully.")

State space dimension: 11
Training and trading environments created successfully.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## Agent training

### Subtask:
Initialize and train a DRL agent (e.g., A2C, PPO, DDPG) using Stable Baselines3 on the training data.


**Reasoning**:
Import the necessary DRL agent classes from stable_baselines3 and train the agent based on the config dictionary.



In [20]:
# Import the selected DRL agent class
if config['AGENT'] == 'a2c':
    from stable_baselines3 import A2C
    Agent = A2C
elif config['AGENT'] == 'ppo':
    from stable_baselines3 import PPO
    Agent = PPO
elif config['AGENT'] == 'ddpg':
    from stable_baselines3 import DDPG
    Agent = DDPG
else:
    raise ValueError(f"Agent {config['AGENT']} not supported.")

# Prepare parameters for the agent constructor, keeping only accepted arguments
agent_params = {k: v for k, v in config['ERL_PARAMS'].items() if k in ['learning_rate', 'n_steps', 'batch_size', 'gamma', 'gae_lambda', 'clip_range', 'clip_range_vf', 'normalize_advantage', 'ent_coef', 'vf_coef', 'max_grad_norm', 'use_sde', 'sde_sample_freq', 'enable_experiencing_repay', 'target_kl', 'create_eval_env', 'policy_kwargs', 'verbose', 'seed', 'device', '_init_setup_model']}

# Instantiate the agent
# We pass the env_train which is already a VecEnv (DummyVecEnv in this case)
# We unpack the filtered agent_params dictionary as keyword arguments
model = Agent("MlpPolicy", env_train, verbose=0, **agent_params)

# Train the agent
print(f"Training agent {config['AGENT']} for {config['TRAIN_STEPS']} steps...")
model.learn(total_timesteps=config['TRAIN_STEPS'])

# Print a message indicating training is complete
print("Agent training complete.")

model.save("ppo_tsm_model.zip")

print("Agent training complete and model saved.")

Training agent ppo for 200000 steps...
day: 2013, episode: 5
begin_total_asset: 100000.00
end_total_asset: 286546.40
total_reward: 186546.40
total_cost: 6148.31
total_trades: 1981
Sharpe: 0.683


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


day: 2013, episode: 10
begin_total_asset: 100000.00
end_total_asset: 181904.15
total_reward: 81904.15
total_cost: 5860.37
total_trades: 1992
Sharpe: 0.430


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


day: 2013, episode: 15
begin_total_asset: 100000.00
end_total_asset: 244306.51
total_reward: 144306.51
total_cost: 6034.39
total_trades: 1991
Sharpe: 0.575


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


day: 2013, episode: 95
begin_total_asset: 100000.00
end_total_asset: 390174.36
total_reward: 290174.36
total_cost: 4622.79
total_trades: 1994
Sharpe: 0.720


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Agent training complete.
Agent training complete and model saved.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [21]:
# Backtesting Cell

# Import necessary libraries for backtesting
from finrl.plot import backtest_stats, backtest_plot
import matplotlib.pyplot as plt # Import matplotlib for plotting
import pandas as pd # Import pandas to ensure Series is available
import numpy as np # Import numpy
from datetime import datetime # Import datetime for index creation

# Ensure the trading environment is reset for the backtest
# The env_trade was created in the Environment Creation Cell (a2ac2c9a)
# The trained model was created in the Agent Training Cell (d0980032)

obs = env_trade.reset()

# Lists to store account value during backtesting
account_value_history = []

print("Running backtesting simulation manually...")

# Iterate through the trading environment
# We will use the env_trade (DummyVecEnv) directly for stepping
# The underlying env can still be accessed to get metrics
env = env_trade.envs[0] # Access the underlying StockTradingEnv


done = False
# Get the initial observation from the VecEnv
obs = env_trade.reset()

# Ensure obs is a numpy array before passing to predict
# DummyVecEnv.reset() usually returns a numpy array directly, but adding a check for robustness
if isinstance(obs, tuple):
    obs = obs[0]

# Collect the initial account value from the environment's asset_memory
# The asset_memory is likely initialized with the initial_amount
# Append the initial account value at the start of the backtest period
account_value_history.append(env.asset_memory[0])


# Loop through the trading days
# The number of trading days is the number of unique dates in the trade_data
num_trading_days = len(trade_data['date'].unique())

# Loop through the trading days/steps
for i in range(num_trading_days):
    # Use the trained model to predict the action
    action, _states = model.predict(obs, deterministic=True)

    # Step the environment with the predicted action using the VecEnv
    obs, reward, done, info = env_trade.step(action)

    # Handle potential tuple observations from newer Gym versions after stepping
    if isinstance(obs, tuple):
        obs = obs[0]

    # Append the account value after the step
    # Ensure asset_memory has at least one element before accessing the last one
    if env.asset_memory:
         account_value_history.append(env.asset_memory[-1])
    else:
         # Fallback if asset_memory is unexpectedly empty
         account_value_history.append(account_value_history[-1] if account_value_history else env.initial_amount)

    # If done is True before the loop finishes (shouldn't happen in a full backtest), break
    if done:
        print(f"Backtesting loop finished early at step {i}.")
        break

# After the loop, the account_value_history should have num_trading_days + 1 entries
# (initial + value after each step).

# Create the index for the account value series
# It should be the trade start date + all trade dates
trade_start_date_str = config['TRADE_START_DATE']
trade_start_date = pd.Timestamp(trade_start_date_str) # Use pandas.Timestamp

trade_dates_full = trade_data['date'].unique()
# Convert trade dates to pandas.Timestamp objects
trade_dates_ts = [pd.Timestamp(d) for d in trade_dates_full]

# The full index includes the start date and all trading dates as pandas.Timestamp
full_index = [trade_start_date] + trade_dates_ts

# Ensure the length of the index matches the length of the history
if len(full_index) != len(account_value_history):
    print(f"Warning: Length of index ({len(full_index)}) does not match length of account value history ({len(account_value_history)}).")
    # If lengths mismatch, we cannot create the series correctly.
    # In a real scenario, you would investigate why the history length is incorrect.
    # For now, we will print a warning and skip statistics/plotting.
    account_value_series = None
    daily_return_series = None
else:
    # Create the account value series
    account_value_series = pd.Series(account_value_history, index=full_index)

    # Calculate daily returns from the account value series
    daily_return_series = account_value_series.pct_change().dropna()

    # Create a DataFrame from the account value series for backtest_stats and backtest_plot
    # These functions expect a DataFrame with an 'account_value' column
    results_df = pd.DataFrame({'account_value': account_value_series})

    # Reset the index to make the date a column named 'index' or 'date'
    results_df = results_df.reset_index()

    # Rename the index column to 'date' as expected by backtest_stats/plot
    results_df.rename(columns={'index': 'date'}, inplace=True)

# Check if we successfully created the series and DataFrame
if 'results_df' in locals() and results_df is not None and len(results_df) > 1: # Need at least two points for returns

    # Generate backtest statistics
    print("\nBacktest Statistics:")
    # Pass the DataFrame to backtest_stats
    display(backtest_stats(results_df))

    # Generate backtest plot
    print("\nBacktest Plot:")
    # Pass the DataFrame to backtest_plot
    backtest_plot(results_df, baseline_ticker = str(config['TICKER_LIST'][0]), baseline_start = config['TRADE_START_DATE'], baseline_end = config['TRADE_END_DATE'])

    print("\nBacktesting complete.")

else:
    print("Not enough account value history collected (need at least 2 points) or index mismatch occurred for statistics and plotting.")
    print(f"Collected {len(account_value_history)} data points.")

Running backtesting simulation manually...
Backtesting loop finished early at step 662.

Backtest Statistics:
Annual return         -3.330669e-16
Cumulative returns    -8.881784e-16
Annual volatility      5.453172e-01
Sharpe ratio           3.930258e-01
Calmar ratio          -5.105869e-16
Stability              8.712009e-01
Max drawdown          -6.523216e-01
Omega ratio            1.099372e+00
Sortino ratio          4.633944e-01
Skew                            NaN
Kurtosis                        NaN
Tail ratio             1.157371e+00
Daily value at risk   -6.785302e-02
dtype: float64


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,0
Annual return,-3.330669e-16
Cumulative returns,-8.881784e-16
Annual volatility,5.453172e-01
Sharpe ratio,3.930258e-01
Calmar ratio,-5.105869e-16
Stability,8.712009e-01
Max drawdown,-6.523216e-01
Omega ratio,1.099372e+00
Sortino ratio,4.633944e-01
Skew,NaN


[*********************100%***********************]  1 of 1 completed


Backtest Plot:
Shape of DataFrame:  (663, 8)


...

Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,65.23,2025-07-24,2025-08-25,NaT,NaN
1,36.81,2025-01-22,2025-04-07,2025-06-25,111
2,22.56,2024-07-09,2024-08-02,2024-10-10,68
3,20.78,2023-06-13,2023-09-25,2024-01-17,157
4,15.60,2023-02-13,2023-04-25,2023-05-24,73


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Stress Events,mean,min,max
Covid,0.09%,-63.76%,12.29%


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



Backtesting complete.
